# Generate UUID report on all public datasets
This notebook builds a table that has two important fields

* `has_uuids` - False if there are no UUIDs present in the UUID-API for this dataset
* `number_of_uuids` - the number of UUIDs in the UUID-API db.

In [15]:
import hubmapbags
from datetime import datetime
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from warnings import warn as warning

token = 'token'
instance = 'prod' #default instance is test

In [16]:
# get assay types
assay_names = hubmapbags.get_assay_types()

In [17]:
report = pd.DataFrame()
for assay_name in assay_names:
    print(assay_name)
    datasets = pd.DataFrame(hubmapbags.get_hubmap_ids( assay_name=assay_name, token=token ))

    if datasets.empty:
        continue
    
    #clean up
    datasets = datasets[(datasets['data_type'] != 'image_pyramid')]
    datasets = datasets[(datasets['status'] == 'Published')]

    datasets['has_uuids'] = None
    datasets['number_of_uuids'] = None
    for index, datum in tqdm(datasets.iterrows()):
        datasets.loc[index, 'number_of_uuids'] = hubmapbags.uuids.get_number_of_uuids( datum['hubmap_id'], token=token )

        if datasets.loc[index, 'number_of_uuids'] == 0:
            datasets.loc[index, 'has_uuids'] = False
        else:
            datasets.loc[index, 'has_uuids'] = True
    
    if report.empty:
        report = datasets
    else:
        report = pd.concat( [report, datasets] )

AF


33it [00:10,  3.02it/s]


ATACseq-bulk


16it [00:04,  3.76it/s]


cell-dive


...

0it [00:00, ?it/s]


LC-MS_top_down


10it [00:02,  3.69it/s]


MS_top_down


10it [00:03,  3.28it/s]


In [18]:
from datetime import datetime
now = datetime.now() 

directory = 'uuid-data-report'

if not Path(directory).exists():
    Path(directory).mkdir()
report.to_csv( directory + '/' + str(now.strftime('%Y%m%d')) + '.tsv', sep='\t', index=False )